# imports, downloads and environment checks

In [ ]:
!pip install -qq ultralytics roboflow kagglehub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.5/974.5 kB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import matplotlib.pyplot as plt
import os
import re
import random
import shutil
import yaml
import ultralytics
from ultralytics import YOLO
import kagglehub
import numpy as np
import torch
import cv2
from pathlib import Path
from typing import Dict

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
ultralytics.checks()

Ultralytics 8.3.107 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 41.1/235.7 GB disk)


In [ ]:
print(f"Is CUDA available: {torch.cuda.is_available()}")
print(f"Current device: {torch.cuda.current_device()}")
print(f"Number of GPUs: {torch.cuda.device_count()}")

Is CUDA available: True
Current device: 0
Number of GPUs: 1


In [ ]:
YOLO("yolov8s.pt").export()
YOLO("yolov8n.pt").export()

100%|██████████| 21.5M/21.5M [00:00<00:00, 87.4MB/s]


Ultralytics 8.3.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)
YOLOv8s summary (fused): 72 layers, 11,156,544 parameters, 0 gradients, 28.6 GFLOPs

PyTorch: starting from 'yolov8s.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (21.5 MB)

TorchScript: starting export with torch 2.6.0+cu124...
TorchScript: export success ✅ 3.1s, saved as 'yolov8s.torchscript' (43.0 MB)

Export complete (5.3s)
Results saved to /content
Predict:         yolo predict task=detect model=yolov8s.torchscript imgsz=640  
Validate:        yolo val task=detect model=yolov8s.torchscript imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'yolov8s.torchscript'

#Load in datasets, merge, clean titles and move into google drive


In [ ]:
path1 = kagglehub.dataset_download("arkadiyhacks/drinking-waste-classification")

print("Path to dataset files:", path1)

Path to dataset files: /kaggle/input/drinking-waste-classification


In [ ]:
global errorcount
errorcount=0

datapath = path1+"/Images_of_Waste/YOLO_imgs"
#dataout = "/content/drive/MyDrive/ENDG 511/Datasets"
dataout=""
train_split=0.6
class_names=["AluCan","Glass","HDPEM","PET"]

os.makedirs("Datasets",exist_ok=True)
for split in ["train","val"]:
  os.makedirs(os.path.join(dataout,split,"images"),exist_ok=True)
  os.makedirs(os.path.join(dataout,split,"labels"),exist_ok=True)

images=[f for f in os.listdir(datapath) if f.endswith(".jpg")]
random.shuffle(images)
train_images=images[:int(len(images)*train_split)]
val_images=images[int(len(images)*train_split):]

def copy_files(image_list, split):
  global errorcount
  for image_file in image_list:
      txt_file = image_file.replace(".jpg", ".txt")

      src_img = os.path.join(datapath, image_file)
      src_lbl = os.path.join(datapath, txt_file)

      dst_img = os.path.join(dataout, split, "images", image_file)
      dst_lbl = os.path.join(dataout, split, "labels", txt_file)

      #some of the dataset doesnt have txt files, check if they do and then copy if yes
      if os.path.exists(src_lbl):
        shutil.copy2(src_img, dst_img)
        shutil.copy2(src_lbl, dst_lbl)
      else:
        print(f"File {dst_lbl} does not exist")
        errorcount+=1


copy_files(train_images, "train")
copy_files(val_images, "val")
print(f"Error count: {errorcount}")

data_yaml = {
    "train": os.path.abspath(os.path.join(dataout, "train", "images")),
    "val": os.path.abspath(os.path.join(dataout, "val", "images")),
    "nc": len(class_names),
    "names": class_names
}

with open(os.path.join(dataout, "data.yaml"), "w") as f:
    yaml.dump(data_yaml, f)

Path to dataset files: /kaggle/input/drinking-waste-classification
File train/labels/Glass670.txt does not exist
File train/labels/Glass664.txt does not exist
File train/labels/Glass666.txt does not exist
File val/labels/Glass672.txt does not exist
File val/labels/Glass668.txt does not exist
Error count: 5


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="xzKghLeum1Ja76dgQSfS")
project = rf.workspace("endg511-ph5py").project("beverage-containers-3atxb-kbkff")
version = project.version(2)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Beverage-Containers-2 in yolov8:: 100%|██████████| 31284/31284 [00:06<00:00, 5013.30it/s]


In [ ]:
# load yaml file and return the data as a dictionary
def load_yaml(path: Path) -> Dict:
    with open(path, 'r') as f:
        return yaml.safe_load(f)

# save dict to yaml
def save_yaml(data: Dict, path: Path):
    with open(path, 'w') as f:
        yaml.safe_dump(data, f, sort_keys=False)

# copy images and label files from original to new directory, combine classes as needed
def copy_and_remap_labels(src_dir: Path, dst_dir: Path, index_map: Dict[int, int]):
    image_dir = src_dir / "images"
    label_dir = src_dir / "labels"

    dst_img_dir = dst_dir / "images"
    dst_lbl_dir = dst_dir / "labels"
    dst_img_dir.mkdir(parents=True, exist_ok=True)
    dst_lbl_dir.mkdir(parents=True, exist_ok=True)

    for img_file in image_dir.glob("*.*"):
        shutil.copy(img_file, dst_img_dir / img_file.name)

    for lbl_file in label_dir.glob("*.txt"):
        new_lines = []
        with open(lbl_file, 'r') as f:
            for line in f:
                if line.strip():
                    parts = line.strip().split()
                    class_id = int(parts[0])
                    new_class_id = index_map[class_id]
                    parts[0] = str(new_class_id)
                    new_lines.append(" ".join(parts))

        with open(dst_lbl_dir / lbl_file.name, 'w') as f:
            f.write("\n".join(new_lines) + "\n")

# pipeline manager function
def merge_yaml_files_and_process(
    file1_path: str,
    file2_path: str,
    output_path: str,
    dataset1_root: str,
    dataset2_root: str,
    merged_root: str,
    cleaned_yaml_output: str,
    cleaned_dataset_output: str
):
    # ensure paths are properly formatted
    file1 = Path(file1_path)
    file2 = Path(file2_path)
    out_file = Path(output_path)
    # load yaml files
    data1 = load_yaml(file1)
    data2 = load_yaml(file2)
    # get and merge class names
    names1 = data1.get('names', [])
    names2 = data2.get('names', [])
    merged_names = list(dict.fromkeys(names1 + names2))
    # create index maps to understand naming order
    index_map1 = {i: merged_names.index(name) for i, name in enumerate(names1)}
    index_map2 = {i: merged_names.index(name) for i, name in enumerate(names2)}

    roboflow = data1.get('roboflow') or data2.get('roboflow')

    merged_yaml = {
        'names': merged_names,
        'nc': len(merged_names),
        'roboflow': roboflow,
        'train': str(Path(merged_root) / "train" / "images"),
        'val': str(Path(merged_root) / "val" / "images"),
        'test': str(Path(merged_root) / "test" / "images")
    }
    # move all files to proper directories
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    save_yaml(merged_yaml, out_file)

    copy_and_remap_labels(Path(dataset1_root) / "train", Path(merged_root) / "train", index_map1)
    copy_and_remap_labels(Path(dataset2_root) / "train", Path(merged_root) / "train", index_map2)

    copy_and_remap_labels(Path(dataset1_root) / "val", Path(merged_root) / "val", index_map1)
    copy_and_remap_labels(Path(dataset2_root) / "val", Path(merged_root) / "val", index_map2)

    copy_and_remap_labels(Path(dataset1_root) / "test", Path(merged_root) / "test", index_map1)
    copy_and_remap_labels(Path(dataset2_root) / "test", Path(merged_root) / "test", index_map2)
    # clean up yaml and reorganize names
    index_map = clean_and_remap_yaml(output_path, cleaned_yaml_output)

    process_dataset_with_cleaned_yaml_internal(index_map, merged_root, cleaned_dataset_output)

#clean yaml with proper names  and save
def clean_and_remap_yaml(yaml_path: str, output_yaml_path: str):
    path = Path(yaml_path)
    data = load_yaml(path)

    original_names = data['names']
    alias_map = {
        'AluCan': 'ALUCAN',
        'Glass': 'bottle-glass',
        'HDPEM': 'HDPEM',
        'PET': 'PET',
        'bottle-plastic': 'PET'
    }
    drop_classes = {
        'cup-disposable', 'cup-handle', 'glass-normal', 'glass-wine', 'gym bottle'
    }

    new_names = []
    remap_dict = {}
    for idx, name in enumerate(original_names):
        std_name = alias_map.get(name, name)
        if std_name in drop_classes:
            continue
        if std_name not in new_names:
            new_names.append(std_name)
        remap_dict[idx] = new_names.index(std_name)

    data['names'] = new_names
    data['nc'] = len(new_names)
    save_yaml(data, Path(output_yaml_path))
    return remap_dict

# remap data
def process_dataset_with_cleaned_yaml_internal(index_map: Dict[int, int], dataset_root: str, output_root: str):
    dataset_root = Path(dataset_root)
    output_root = Path(output_root)

    print("Processing started ")
    copy_and_remap_labels(dataset_root / "train", output_root / "train", index_map)
    copy_and_remap_labels(dataset_root / "val", output_root / "val", index_map)
    copy_and_remap_labels(dataset_root / "test", output_root / "test", index_map)



In [ ]:

# Run the merge
merge_yaml_files_and_process(
    "/content/Beverage-Containers-2/data.yaml",
    "/content/data.yaml",
    "/content/drive/MyDrive/ENDG 511/merged_dataset6/merged.yaml",
    "/content/Beverage-Containers-2",
    "/content",
    "/content/drive/MyDrive/ENDG 511/merged_dataset6",
    "/content/drive/MyDrive/ENDG 511/cleaned7/cleaned.yaml",
    "/content/drive/MyDrive/ENDG 511/cleaned7"
)

#Fine Tuning

In [ ]:
# Define paths for saving
model_folder = "/content/drive/MyDrive/ENDG 511/Models"
results_folder = "/content/drive/MyDrive/ENDG 511/Models/Results"

#create train and save function to be used in looping fine tuning
def trainandsave(data, epochs, lr, optimizer, dropout,augment,freeze):
  # List all existing model files matching 'modelX.pt'
  existing_models = [f for f in os.listdir(model_folder) if re.match(r'model\d+\.pt', f)]

  # Extract numbers from filenames and find the highest model number
  existing_numbers = [int(re.search(r'\d+', f).group()) for f in existing_models]
  next_number = max(existing_numbers, default=0) + 1  # Default to 1 if no models exist
  # Train the YOLO model and store results in a specific folder
  tempmodel = YOLO("/content/yolov8n.pt")
  results = tempmodel.train(
      data=data,
      epochs=epochs,
      plots=True,
      batch=-1,
      project=results_folder,
      name=f"results{next_number}",
      seed=42,
      lr0 = lr,
      optimizer = optimizer,
      overlap_mask=True,
      dropout = dropout,
      augment = augment,
      freeze = freeze
      )

  # Path to the best-trained model inside the results folder
  trained_model_path = os.path.join(results_folder, f"results{next_number}", "weights", "best.pt")
  next_model_name = f"model{next_number}.pt"
  next_model_path = os.path.join(model_folder, next_model_name)

  # Move the best model to the new location
  if os.path.exists(trained_model_path):
      shutil.copy2(trained_model_path, next_model_path)
      print(f"Model saved as: {next_model_path}")
  else:
      print(f"Error: Trained model not found at {trained_model_path}. Check if training completed successfully.")
  return results, next_number

In [ ]:
#example of looping code used throughout fine tuning
lrs = [0.0005,0.001,0.005]
epochs = [50, 75, 100]
optimizers = ["adamW", "adam"]
dropout = [0,0.2,.4]
augments = [True,False]
freeze = [0,5,7,8]

data = "/content/drive/MyDrive/ENDG 511/cleaned2/cleaned.yaml"

for e in epochs:
  for l in lrs:
    for o in optimizers:
      for d in dropout:
        for a in augments:
          for f in freeze:
            print(next_number)
            results, next_number = trainandsave(data, e, l, o, d,a,f)
            file = open("/content/drive/MyDrive/ENDG 511/Models/Resultslog.txt", 'a')
            file.write(f"\nmodel{next_number}|{e}|{l}|{o}|{results.box.map}|dropout:{d}|augment:{a}")
            file.close()

52
Ultralytics 8.3.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=/content/yolov8s.pt, data=/content/drive/MyDrive/ENDG 511/cleaned2/cleaned.yaml, epochs=50, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/ENDG 511/Models/Results, name=results5239, exist_ok=False, pretrained=True, optimizer=adamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=8, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.4, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt

100%|██████████| 755k/755k [00:00<00:00, 43.0MB/s]


Overriding model.yaml nc=80 with nc=7

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 157MB/s]


AMP: checks passed ✅


KeyboardInterrupt: 

In [ ]:
# getting closer to final runthrough
results, next_number = trainandsave(data,75,0.00075,"adamw",0.4,True,8)
file = open("/content/drive/MyDrive/ENDG 511/Models/Resultslog.txt", 'a')
file.write(f"\nmodel{next_number}|{e}|{l}|{o}|{results.box.map}|dropout:{d}|augment:{a}")
file.close()

New https://pypi.org/project/ultralytics/8.3.103 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.102 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=/content/yolov8s.pt, data=/content/drive/MyDrive/ENDG 511/cleaned2/cleaned.yaml, epochs=75, time=None, patience=100, batch=120, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/ENDG 511/Models/Results, name=results5232, exist_ok=False, pretrained=True, optimizer=adamw, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=8, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.4, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augmen

Traceback (most recent call last):
  File "/usr/lib/python3.11/multiprocessing/queues.py", line 239, in _feed
    reader_close()
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 178, in close
    self._close()
  File "/usr/lib/python3.11/multiprocessing/connection.py", line 377, in _close
    _close(self._handle)
OSError: [Errno 9] Bad file descriptor


Freezing layer 'model.0.conv.weight'
Freezing layer 'model.0.bn.weight'
Freezing layer 'model.0.bn.bias'
Freezing layer 'model.1.conv.weight'
Freezing layer 'model.1.bn.weight'
Freezing layer 'model.1.bn.bias'
Freezing layer 'model.2.cv1.conv.weight'
Freezing layer 'model.2.cv1.bn.weight'
Freezing layer 'model.2.cv1.bn.bias'
Freezing layer 'model.2.cv2.conv.weight'
Freezing layer 'model.2.cv2.bn.weight'
Freezing layer 'model.2.cv2.bn.bias'
Freezing layer 'model.2.m.0.cv1.conv.weight'
Freezing layer 'model.2.m.0.cv1.bn.weight'
Freezing layer 'model.2.m.0.cv1.bn.bias'
Freezing layer 'model.2.m.0.cv2.conv.weight'
Freezing layer 'model.2.m.0.cv2.bn.weight'
Freezing layer 'model.2.m.0.cv2.bn.bias'
Freezing layer 'model.3.conv.weight'
Freezing layer 'model.3.bn.weight'
Freezing layer 'model.3.bn.bias'
Freezing layer 'model.4.cv1.conv.weight'
Freezing layer 'model.4.cv1.bn.weight'
Freezing layer 'model.4.cv1.bn.bias'
Freezing layer 'model.4.cv2.conv.weight'
Freezing layer 'model.4.cv2.bn.weig

train: Scanning /content/drive/MyDrive/ENDG 511/cleaned2/train/labels.cache... 5005 images, 2445 backgrounds, 9 corrupt: 100%|██████████| 7450/7450 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned2/train/images/AluCan39.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0469]
train: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned2/train/images/AluCan48.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.002]
train: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned2/train/images/AluCan87.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0339]
train: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned2/train/images/Glass436.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0078]
train: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned2/train/images/Glass471.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0052]
train: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned2/train/images/Glass84.jpg: ignoring corrupt image/label: non-normalized 

...

      75/75      21.5G     0.3194     0.1759     0.8188          0        640: 100%|██████████| 63/63 [00:35<00:00,  1.79it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:07<00:00,  1.13it/s]

                   all       1921       2018      0.986      0.983      0.991      0.871



75 epochs completed in 0.962 hours.
Optimizer stripped from /content/drive/MyDrive/ENDG 511/Models/Results/results5232/weights/last.pt, 22.5MB
Optimizer stripped from /content/drive/MyDrive/ENDG 511/Models/Results/results5232/weights/best.pt, 22.5MB

Validating /content/drive/MyDrive/ENDG 511/Models/Results/results5232/weights/best.pt...
Ultralytics 8.3.102 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,128,293 parameters, 0 gradients, 28.5 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:10<00:00,  1.22s/it]


                   all       1921       2018      0.985      0.988       0.99      0.871
                ALUCAN        460        479      0.992      0.976      0.992      0.787
                 HDPEM        400        400      0.985      0.998      0.992      0.883
                   PET        595        595      0.992       0.98      0.994      0.916
          bottle-glass        481        544      0.972      0.998      0.981      0.897
Speed: 0.1ms preprocess, 1.9ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /content/drive/MyDrive/ENDG 511/Models/Results/results5232
Error: Trained model not found at /content/drive/MyDrive/ENDG 511/Models/Results/results52/weights/best.pt. Check if training completed successfully.


In [ ]:
# final model run
data = "/content/drive/MyDrive/ENDG 511/cleaned7/cleaned.yaml"

results, next_number = trainandsave(data,125,0.00075,"adamw",0.4,True,8)
file = open("/content/drive/MyDrive/ENDG 511/Models/Resultslog.txt", 'a')
file.write(f"\nmodel{next_number}|{125}|0.00075|adamw|{results.box.map}|dropout:0.4|augment:True")
file.close()

New https://pypi.org/project/ultralytics/8.3.104 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: task=detect, mode=train, model=/content/yolov8s.pt, data=/content/drive/MyDrive/ENDG 511/cleaned7/cleaned.yaml, epochs=125, time=None, patience=100, batch=275, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/ENDG 511/Models/Results, name=results5250, exist_ok=False, pretrained=True, optimizer=adamw, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=8, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.4, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augme

train: Scanning /content/drive/MyDrive/ENDG 511/cleaned7/train/labels.cache... 6414 images, 10173 backgrounds, 8 corrupt: 100%|██████████| 16588/16588 [00:00<?, ?it/s]

train: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned7/train/images/AluCan39.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0469]
train: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned7/train/images/AluCan48.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.002]
train: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned7/train/images/AluCan55.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.013]
train: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned7/train/images/Glass435.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0078]
train: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned7/train/images/Glass471.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0052]
train: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned7/train/images/PET1,550.jpg: ignoring corrupt image/label: non-normalized


val: Scanning /content/drive/MyDrive/ENDG 511/cleaned7/val/labels.cache... 1925 images, 0 backgrounds, 6 corrupt: 100%|██████████| 1925/1925 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned7/val/images/AluCan87.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0339]
val: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned7/val/images/Glass385.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0052]
val: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned7/val/images/Glass436.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0078]
val: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned7/val/images/Glass84.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.026]
val: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned7/val/images/HDPEM22.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0052]
val: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned7/val/images/PET1,620.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordina

...

    125/125      30.8G     0.3117     0.1693     0.8306         42        640: 100%|██████████| 61/61 [01:16<00:00,  1.26s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:07<00:00,  1.99s/it]

                   all       1919       2027      0.991      0.992      0.994      0.874



125 epochs completed in 3.351 hours.
Optimizer stripped from /content/drive/MyDrive/ENDG 511/Models/Results/results5250/weights/last.pt, 22.5MB
Optimizer stripped from /content/drive/MyDrive/ENDG 511/Models/Results/results5250/weights/best.pt, 22.5MB

Validating /content/drive/MyDrive/ENDG 511/Models/Results/results5250/weights/best.pt...
Ultralytics 8.3.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,127,132 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:20<00:00,  5.01s/it]


                   all       1919       2027      0.991      0.992      0.992      0.872
                ALUCAN        468        491      0.988      0.984       0.99      0.782
                 HDPEM        393        394      0.997      0.997      0.995      0.885
                   PET        578        578       0.99      0.995      0.995      0.923
          bottle-glass        494        564      0.988      0.992      0.986      0.898
Speed: 0.1ms preprocess, 1.6ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to /content/drive/MyDrive/ENDG 511/Models/Results/results5250
Error: Trained model not found at /content/drive/MyDrive/ENDG 511/Models/Results/results52/weights/best.pt. Check if training completed successfully.


# Evaluate model

In [ ]:
model = YOLO("/content/drive/MyDrive/ENDG 511/Models/Results/results5250/weights/best.pt")
results = model.val()
print(results.results_dict)

Ultralytics 8.3.103 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
Model summary (fused): 72 layers, 11,127,132 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /content/drive/MyDrive/ENDG 511/cleaned7/val/labels.cache... 1925 images, 0 backgrounds, 6 corrupt: 100%|██████████| 1925/1925 [00:00<?, ?it/s]

val: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned7/val/images/AluCan87.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0339]
val: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned7/val/images/Glass385.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0052]
val: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned7/val/images/Glass436.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0078]
val: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned7/val/images/Glass84.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [      1.026]
val: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned7/val/images/HDPEM22.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0052]
val: WARNING ⚠️ /content/drive/MyDrive/ENDG 511/cleaned7/val/images/PET1,620.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordina


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 120/120 [00:23<00:00,  5.14it/s]


                   all       1919       2027      0.992      0.989      0.994      0.873
                ALUCAN        468        491      0.992      0.977      0.994      0.788
                 HDPEM        393        394      0.999      0.997      0.995      0.888
                   PET        578        578      0.997      0.991      0.995      0.921
          bottle-glass        494        564       0.98      0.991      0.991      0.895
Speed: 0.2ms preprocess, 1.2ms inference, 0.0ms loss, 0.9ms postprocess per image
Results saved to runs/detect/val
{'metrics/precision(B)': np.float64(0.9918954957199098), 'metrics/recall(B)': np.float64(0.9891121026458023), 'metrics/mAP50(B)': np.float64(0.9937018334927528), 'metrics/mAP50-95(B)': np.float64(0.8728643791772196), 'fitness': np.float64(0.8849481246087729)}


#Compile Results


In [ ]:
directory = "/content/drive/MyDrive/ENDG 511/Models/Results"
DIR_PATTERN = re.compile(r"^results(\d{1,4})$")

final_data = []

for subdir in os.listdir(directory):
    match = DIR_PATTERN.match(subdir)
    if not match:
        continue

    result_id = match.group(1)
    result_folder = os.path.join(directory, subdir)
    csv_path = os.path.join(result_folder, "results.csv")
    yaml_path = os.path.join(result_folder, "args.yaml")

    if not os.path.isfile(csv_path) or not os.path.isfile(yaml_path):
        continue

    try:
        df = pd.read_csv(csv_path)
        if df.empty:
            continue

        last_row = df.iloc[-1]
        epoch = last_row.iloc[0]
        box_loss = last_row.iloc[2]
        cls_loss = last_row.iloc[3]

        with open(yaml_path, 'r') as f:
            args = yaml.safe_load(f)

        # Extract specific args
        epochs = args.get("epochs")
        model = args.get("model")
        optimizer = args.get("optimizer")
        mask_ratio = args.get("mask_ratio")
        close_mosaic = args.get("close_mosaic")
        split = args.get("split")
        lr0 = args.get("lr0")
        dropout = args.get("dropout")
        augment = args.get("augment")
        freeze = args.get("freeze")

        final_data.append([
            result_id, epoch, box_loss, cls_loss,
            epochs, model, optimizer, mask_ratio,
            close_mosaic, split, lr0, dropout,
            augment, freeze
        ])

    except Exception as e:
        print(f"Error processing {subdir}: {e}")

columns = [
    "id", "epoch", "box_loss", "cls_loss",
    "arg_epochs", "arg_model", "arg_optimizer",
    "arg_mask_ratio", "arg_close_mosaic", "arg_split",
    "arg_lr0", "arg_dropout", "arg_augment", "arg_freeze"
]

results_df = pd.DataFrame(final_data, columns=columns)
results_df.to_csv("compiled_results.csv", index=False)

print("✅ Compilation complete. Output saved to compiled_results.csv")

✅ Compilation complete. Output saved to compiled_results.csv
